**Quiz 6 of Frequentist Statistic by Ihor Markevych.**

In [1]:
library(car)
library(pastecs)
library(WRS2)
library(gmodels)
library(MASS)

Loading required package: carData


In [2]:
d <- read.csv('Final_Data_University_IM.csv')
d$CRIMINAL_RECORD  <- as.factor(d$CRIMINAL_RECORD)
d$ADMITTED <- as.factor(d$ADMITTED)

In [3]:
head(d)

GPA,HOURS_STUDIED_FOR_ENRANCE_EXAM,GRE,RACE,CRIMINAL_RECORD,ADMITTED
2.80,50,264,Human,0,0
2.80,50,260,Dwarf,1,0
3.28,59,337,Human,0,1
2.80,50,287,Elf,0,0
2.80,44,281,Elf,0,0
2.90,51,270,Elf,0,0


In [4]:
summary(d)

      GPA        HOURS_STUDIED_FOR_ENRANCE_EXAM      GRE           RACE    
 Min.   :2.800   Min.   :33.00                  Min.   :260.0   Dwarf:244  
 1st Qu.:2.800   1st Qu.:47.00                  1st Qu.:270.0   Elf  :250  
 Median :2.800   Median :50.00                  Median :281.0   Human:258  
 Mean   :2.928   Mean   :50.13                  Mean   :288.9   Orc  :127  
 3rd Qu.:2.990   3rd Qu.:54.00                  3rd Qu.:305.0   Troll:121  
 Max.   :4.000   Max.   :69.00                  Max.   :340.0              
 CRIMINAL_RECORD ADMITTED
 0:870           0:799   
 1:130           1:201   
                         
                         
                         
                         

# Question 1
Since we want to check for three way interaction for three categorical variables, we'll go with **loglin**.

In [5]:
#create contingency table 
contingencyTable <- xtabs(~ ADMITTED + CRIMINAL_RECORD + RACE, data = d)

#create saturated model

saturated <- loglm(~ ADMITTED * CRIMINAL_RECORD * RACE, data = contingencyTable)

#remove threeWay interaction
threeWay <- update(saturated, .~. -ADMITTED : CRIMINAL_RECORD : RACE)

#compare models
anova(saturated, threeWay)

LR tests for hierarchical log-linear models

Model 1:
 . ~ ADMITTED + CRIMINAL_RECORD + RACE + ADMITTED:CRIMINAL_RECORD + ADMITTED:RACE + CRIMINAL_RECORD:RACE 
Model 2:
 ~ADMITTED * CRIMINAL_RECORD * RACE 

          Deviance df Delta(Dev) Delta(df) P(> Delta(Dev)
Model 1   3.503252  4                                    
Model 2   0.000000  0   3.503252         4        0.47738
Saturated 0.000000  0   0.000000         0        1.00000

As we can see, there is no significant difference between models that include three way interaction and that does not, so, this interaction is not significant (there is no interaction).

-------------------------------------------------

# Question 2
We have `GPA` and `RACE` - one numeric var, one categorical. For this case we can use **ANOVA**.

In [6]:
leveneTest(d$GPA, d$RACE, center = median)

,Df,F value,Pr(>F)
group,4,1.717966,0.1437455
,995,NA,NA


Levene test not violated, we can go with ANOVA.

In [7]:
m <- aov(GPA ~ RACE, data = d)  
Anova(m, type='III')

,Sum Sq,Df,F value,Pr(>F)
(Intercept),2087.338507,1,38185.553293,0.0000000
RACE,0.375637,4,1.717966,0.1437455
Residuals,54.389727,995,NA,NA


As we can see, $p-value$ is big enough to claim that there is no evidence to say that there is any difference.

-------------

# Question 3
We have `GRE` and `RACE` - one numeric var, one categorical. For this case we can use **ANOVA**.

In [8]:
leveneTest(d$GRE, d$RACE, center = median)

,Df,F value,Pr(>F)
group,4,1.19652,0.3106903
,995,NA,NA


Levene test not violated, we can go with ANOVA.

In [9]:
m <- aov(GRE ~ RACE, data = d)  
Anova(m, type='III')

,Sum Sq,Df,F value,Pr(>F)
(Intercept),20693021.512,1,3.458520e+04,0.0000000
RACE,2062.561,4,8.618134e-01,0.4863286
Residuals,595328.558,995,NA,NA


As we can see, $p-value$ is big enough to claim that there is no evidence to say that there is any difference.

-------------

# Question 4
This should be a **FACTORIAL ANOVA**, since we have two categorical variables (`CRIMINAL_RECORD` and `RACE`) and one numerical (`GPA`).

In [10]:
leveneTest(d$GPA, interaction(d$RACE, d$CRIMINAL_RECORD), center = median)

,Df,F value,Pr(>F)
group,9,2.648314,0.004916428
,990,NA,NA


Levene test violated, we'll go with robust FACTORIAL ANOVA.

In [11]:
mcp2a(GPA ~ RACE + CRIMINAL_RECORD, d, nboot = 200)

Aggregation requires fun.aggregate: length used as default


Call:
mcp2a(formula = GPA ~ RACE + CRIMINAL_RECORD, data = d, nboot = 200)

                        psihat ci.lower ci.upper p-value
RACE1                        0        0        0       0
RACE2                        0        0        0       0
RACE3                        1        1        1       0
RACE4                        1        1        1       0
RACE5                        0        0        0       0
RACE6                        1        1        1       0
RACE7                        1        1        1       0
RACE8                        1        1        1       0
RACE9                        1        1        1       0
RACE10                       0        0        0       0
CRIMINAL_RECORD1             3        3        3       0
RACE1:CRIMINAL_RECORD1       0        0        0       0
RACE2:CRIMINAL_RECORD1       0        0        0       0
RACE3:CRIMINAL_RECORD1       1        1        1       0
RACE4:CRIMINAL_RECORD1       1        1        1       0
RACE5:CRIMIN

All values are small enough to say that there is enough evidence to claim difference.

----------

# Question 5
I believe Question 5 is just rephrased Question 4, so, answer will be same.

---------

# Question 6

Since we have one categorical variable (`ADMITTED`) and one numerical (`HOURS_STUDIED_FOR_ENRANCE_EXAM`), we'll go with **ANOVA** again.



In [12]:
leveneTest(d$HOURS_STUDIED_FOR_ENRANCE_EXAM, d$ADMITTED, center = median)

,Df,F value,Pr(>F)
group,1,5.645061,0.01769256
,998,NA,NA


This is violated, will need to go with non-parametric test. Options here are:
- Wilcox(paired=F)
- K-W
- robust ANOVA

Wilcox(paired=F):

In [13]:
m <- wilcox.test(HOURS_STUDIED_FOR_ENRANCE_EXAM ~ ADMITTED, data = d)

In [14]:
m


	Wilcoxon rank sum test with continuity correction

data:  HOURS_STUDIED_FOR_ENRANCE_EXAM by ADMITTED
W = 15939, p-value < 2.2e-16
alternative hypothesis: true location shift is not equal to 0


Robust ANOVA:

In [15]:
t1way(d$HOURS_STUDIED_FOR_ENRANCE_EXAM ~ d$ADMITTED, tr = .2)

Call:
t1way(formula = d$HOURS_STUDIED_FOR_ENRANCE_EXAM ~ d$ADMITTED, 
    tr = 0.2)

Test statistic: F = 476.3973 
Degrees of freedom 1: 1 
Degrees of freedom 2: 210.01 
p-value: 0 

Explanatory measure of effect size: 0.89 


In [16]:
t1waybt(d$HOURS_STUDIED_FOR_ENRANCE_EXAM ~ d$ADMITTED, tr = .2)

Call:
t1waybt(formula = d$HOURS_STUDIED_FOR_ENRANCE_EXAM ~ d$ADMITTED, 
    tr = 0.2)

Effective number of bootstrap samples was 599.

Test statistic: 476.3973 
p-value: 0 
Variance explained 0.814 
Effect size 0.902 


K-W:

In [17]:
kruskal.test(d$HOURS_STUDIED_FOR_ENRANCE_EXAM ~ d$ADMITTED)


	Kruskal-Wallis rank sum test

data:  d$HOURS_STUDIED_FOR_ENRANCE_EXAM by d$ADMITTED
Kruskal-Wallis chi-squared = 310.69, df = 1, p-value < 2.2e-16


All of those tests gave same result - $p-value$ is small enough to claim significance.

-----------

# Question 7

Since we have two categorical variable (`ADMITTED`, `RACE`), we'll do $\chi ^ 2$-test.



In [18]:
CrossTable(d$ADMITTED, d$RACE, fisher = TRUE, chisq = TRUE, expected = TRUE, sresid = TRUE, format = "SPSS")


   Cell Contents
|-------------------------|
|                   Count |
|         Expected Values |
| Chi-square contribution |
|             Row Percent |
|          Column Percent |
|           Total Percent |
|            Std Residual |
|-------------------------|

Total Observations in Table:  1000 

             | d$RACE 
  d$ADMITTED |    Dwarf  |      Elf  |    Human  |      Orc  |    Troll  | Row Total | 
-------------|-----------|-----------|-----------|-----------|-----------|-----------|
           0 |      184  |      198  |      209  |      111  |       97  |      799  | 
             |  194.956  |  199.750  |  206.142  |  101.473  |   96.679  |           | 
             |    0.616  |    0.015  |    0.040  |    0.894  |    0.001  |           | 
             |   23.029% |   24.781% |   26.158% |   13.892% |   12.140% |   79.900% | 
             |   75.410% |   79.200% |   81.008% |   87.402% |   80.165% |           | 
             |   18.400% |   19.800% |   20.900% |   1

As we can see, $p-value$ is big enough to say that there is no evidence to claim difference.

-----------